In [1]:
from transformers import AutoTokenizer, BertForMaskedLM
from diffusers import DDPMScheduler
import numpy as np
import matplotlib.pyplot as plt
from modeling_diffbert_sample import DiffBertForDiffusion
from configuration_diffbert import DiffBertConfig
import torch
import inspect
from typing import Any, Callable, Dict, List, Optional, Union
from tqdm.auto import tqdm
import numpy as np
import torch.nn.functional as F

    

    
# model(inputs_embeds=inputs_embeds, timesteps=timesteps).logits.shape

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
scheduler = DDPMScheduler(prediction_type="sample", num_train_timesteps=2000)
model = DiffBertForDiffusion.from_pretrained("diffbert-mini-sample-trained").to("cuda")
device = model.device
embedding = BertForMaskedLM.from_pretrained("neuralmind/bert-base-portuguese-cased").to(device)#torch.nn.Embedding(model.config.vocab_size, model.config.hidden_size).to(device)
# embedding.load_state_dict(torch.load('diffbert-mini/embedding_weights.bin'))

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:

def retrieve_timesteps(
    scheduler,
    num_inference_steps: Optional[int] = None,
    device: Optional[Union[str, torch.device]] = None,
    timesteps: Optional[List[int]] = None,
    **kwargs,
):
    """
    Calls the scheduler's `set_timesteps` method and retrieves timesteps from the scheduler after the call. Handles
    custom timesteps. Any kwargs will be supplied to `scheduler.set_timesteps`.

    Args:
        scheduler (`SchedulerMixin`):
            The scheduler to get timesteps from.
        num_inference_steps (`int`):
            The number of diffusion steps used when generating samples with a pre-trained model. If used,
            `timesteps` must be `None`.
        device (`str` or `torch.device`, *optional*):
            The device to which the timesteps should be moved to. If `None`, the timesteps are not moved.
        timesteps (`List[int]`, *optional*):
                Custom timesteps used to support arbitrary spacing between timesteps. If `None`, then the default
                timestep spacing strategy of the scheduler is used. If `timesteps` is passed, `num_inference_steps`
                must be `None`.

    Returns:
        `Tuple[torch.Tensor, int]`: A tuple where the first element is the timestep schedule from the scheduler and the
        second element is the number of inference steps.
    """
    if timesteps is not None:
        accepts_timesteps = "timesteps" in set(inspect.signature(scheduler.set_timesteps).parameters.keys())
        if not accepts_timesteps:
            raise ValueError(
                f"The current scheduler class {scheduler.__class__}'s `set_timesteps` does not support custom"
                f" timestep schedules. Please check whether you are using the correct scheduler."
            )
        scheduler.set_timesteps(timesteps=timesteps, device=device, **kwargs)
        timesteps = scheduler.timesteps
        num_inference_steps = len(timesteps)
    else:
        scheduler.set_timesteps(num_inference_steps, device=device, **kwargs)
        timesteps = scheduler.timesteps
    return timesteps, num_inference_steps

def id_to_one_hot(token_ids, vocab_size=tokenizer.vocab_size):
    one_hot_vectors = []
    for token_id in token_ids:
        # Create a zero-filled array with length equal to vocab_size
        one_hot = torch.zeros(vocab_size)
        # Set the value at the index of the token ID to 1
        one_hot[token_id] = 1
        one_hot_vectors.append(one_hot)
    return torch.stack(one_hot_vectors, dim=0)

def get_max_indices(list_of_tensors):
    max_indices = []
    for tensor in list_of_tensors:
        # Get the index of the maximum value in the tensor
        index = torch.argmax(tensor).item()
        max_indices.append(index)
    return max_indices
# Function to transform vectors back to indices
def vectors_to_indices(vectors, embedding):
    # Calculate cosine similarity between vectors and all embedding weights
    # similarity = torch.matmul(vectors, embedding.weight.T)
    
    # Get the index of the most similar embedding for each vector
    indices = torch.argmax(vectors, dim=-1)
    
    return indices
def sample_text(probabilities, temperature=1.0):
    batch_size, seq_len, vocab_size = probabilities.size()
    flattened_probs = probabilities.view(batch_size * seq_len, -1)
    
    scaled_logits = flattened_probs / temperature
    scaled_probs = F.softmax(scaled_logits, dim=-1)
    
    sampled_indices = torch.multinomial(scaled_probs, 1)
    sampled_token_ids = sampled_indices.view(batch_size, seq_len)
    
    return sampled_token_ids

In [4]:




with torch.no_grad():
    latents = torch.rand((1, 128, 768), device=device)
    print(latents)
    num_inference_steps = 2000
    timesteps=None#[999, 500, 1]
    timesteps, num_inference_steps = retrieve_timesteps(scheduler, num_inference_steps, device, timesteps)
    # print(timesteps)
    for i, t in tqdm(enumerate(timesteps)):
        # expand the latents if we are doing classifier free guidance
        latent_model_input =  latents
        latent_model_input = scheduler.scale_model_input(latent_model_input, t)
        # predict the noise residual
        outputs = model(
            input_embeds=latent_model_input,
            timesteps=t.reshape(1,).to(device),
            # encoder_hidden_states=prompt_embeds,
            # timestep_cond=timestep_cond,
            # cross_attention_kwargs=self.cross_attention_kwargs,
            # added_cond_kwargs=added_cond_kwargs,
            # return_dict=False,
        )
        noise_pred = outputs.last_hidden_state
        latents_final = outputs.logits



        # compute the previous noisy sample x_t -> x_t-1
        # print(scheduler.step(noise_pred, t, latents, return_dict=True))
        step = scheduler.step(noise_pred, t, latents, return_dict=True)#[0]
        latents = step["prev_sample"]
        # latents_final = step["pred_original_sample"]

tensor([[[0.8234, 0.9887, 0.5801,  ..., 0.3357, 0.0893, 0.1092],
         [0.5783, 0.2146, 0.5057,  ..., 0.9016, 0.1550, 0.8159],
         [0.8826, 0.3938, 0.5698,  ..., 0.6492, 0.9796, 0.7409],
         ...,
         [0.2288, 0.3776, 0.8710,  ..., 0.4842, 0.9594, 0.9722],
         [0.3592, 0.2566, 0.0638,  ..., 0.7040, 0.1135, 0.1932],
         [0.4192, 0.2948, 0.1341,  ..., 0.2115, 0.8594, 0.5207]]],
       device='cuda:0')


0it [00:00, ?it/s]

In [5]:
latents_final

tensor([[[ 2.0815, -7.2645, -7.6150,  ..., -7.6257, -6.7353, -7.3195],
         [-1.3201, -8.1812, -9.5754,  ..., -8.7641, -8.3628, -9.2203],
         [ 0.9122, -8.9232, -9.4601,  ..., -9.6429, -8.5498, -9.2966],
         ...,
         [-0.5197, -7.8803, -8.7028,  ..., -8.6381, -8.0927, -9.1499],
         [-0.7567, -7.5069, -8.4107,  ..., -8.3093, -6.2122, -7.5812],
         [ 8.9878, -6.1929, -6.8535,  ..., -6.9554, -6.0324, -6.8638]]],
       device='cuda:0')

In [6]:

top_k_values, top_k_indices = torch.topk(latents_final[0][0], k=5)
print(top_k_values)
print(top_k_indices[0])
tokenizer.decode(top_k_indices)

tensor([16.9424,  4.7998,  3.8280,  3.6118,  3.3611], device='cuda:0')
tensor(101, device='cuda:0')


'[CLS] from a -,'

In [7]:
print(vectors_to_indices(latents_final[0], embedding))
# print(tokenizer.decode(vectors_to_indices(latents[0], embedding)))
print(tokenizer.decode(sample_text(latents_final, 1)[0]))
print(tokenizer.decode(vectors_to_indices(latents_final[0], embedding)))

tensor([ 101, 2043, 2014, 2057, 2038, 2001, 2044, 2012, 2016, 2040, 2001, 2016,
        2001, 2012, 2001, 2012, 2001, 2012, 1037, 2038, 2016, 2013, 2016, 2001,
        2038, 2042, 2048, 2016, 2001, 2016, 2038, 2042, 2016, 2042, 2001, 2042,
        2012, 2043, 2012, 2013, 2016, 2420, 2012, 2038, 2016, 2001, 1037, 2042,
        2025, 2016, 2016, 2001, 2042, 2014, 2016, 2001, 2043, 2042, 1011, 2016,
        2042, 2012, 2038, 2042, 2042, 2001, 2016, 2014, 2012, 2001, 2042, 2012,
        1017, 2043, 2016, 2012, 2037, 2001, 2016, 2074, 2024, 2016, 2012, 2014,
        2001, 2016, 2001, 2042, 2043, 2042, 2013, 2138, 2042, 1011, 2013, 1037,
        2038, 2042, 2001, 2016, 2001, 2001, 2001, 1997, 1037, 2014, 2016, 2096,
        2016, 2016, 2012, 2035, 2038, 2042, 2042, 2012, 2016, 1011, 2014, 2482,
        2072, 2001, 2042, 2012, 2001, 2038, 2014,  102], device='cuda:0')
[CLS] when we we has was after at she who was she was at was at was at a has she from she was has been two she was she has bee

In [1]:
import torch
import torch.nn as nn

# Define the size of vocabulary and embedding dimension
vocab_size = 100  # Example vocabulary size
embedding_dim = 50  # Example embedding dimension size

# Instantiate nn.Embedding module
embedding = nn.Embedding(vocab_size, embedding_dim)

# Random indices for demonstration purposes
indices = torch.tensor([3, 7, 15])  # Example input indices

# Convert indices to vectors using the embedding layer
vectors = embedding(indices)

# Display the vectors corresponding to the input indices
print("Vectors corresponding to input indices:")
print(vectors)

# Function to transform vectors back to indices
def vectors_to_indices(vectors, embedding):
    # Calculate cosine similarity between vectors and all embedding weights
    similarity = torch.matmul(vectors, embedding.weight.T)
    
    # Get the index of the most similar embedding for each vector
    indices = torch.argmax(similarity, dim=1)
    
    return indices

# Convert vectors back to indices
recovered_indices = vectors_to_indices(vectors, embedding)

# Display the indices recovered from vectors
print("\nRecovered indices from vectors:")
print(recovered_indices)

Vectors corresponding to input indices:
tensor([[-7.7292e-02,  1.1234e+00, -1.1162e+00,  9.5290e-01,  7.0411e-01,
          9.3934e-02,  1.2905e-01, -6.1421e-01, -4.7354e-01,  1.8669e+00,
          1.3230e+00,  7.4839e-01,  3.6166e-01, -7.6501e-01, -3.1029e-01,
         -1.3262e+00, -1.2330e+00, -2.1209e-01, -1.2452e+00,  6.3154e-01,
         -4.2177e-01, -6.7838e-01,  1.8145e-01, -2.4687e-01,  4.7213e-01,
          2.9644e-01,  5.5261e-01, -1.4998e+00, -3.2089e-01,  1.9922e+00,
         -2.7300e-01, -1.3218e+00, -2.0146e-01,  1.8222e-02, -1.4948e+00,
         -5.4760e-01, -3.8630e-01, -6.9837e-01, -1.0270e-01,  8.3724e-01,
         -6.1612e-02, -1.1182e+00,  3.0394e+00, -2.8233e-01,  7.6667e-01,
         -2.0013e-01,  1.4309e+00, -4.0717e-01, -7.3446e-01,  8.6851e-02],
        [-8.8197e-01, -8.1627e-01, -9.9473e-01, -2.0596e-01, -4.3363e-01,
         -1.3574e+00,  8.7575e-01,  4.4570e-02,  6.7288e-01, -8.9306e-01,
         -5.2451e-01,  6.8276e-02,  9.4779e-01,  8.5183e-01,  1.8238e+0